#### 네이버 뉴스 sub 페이지 크롤링
    - 한개의 sub 페이지 크롤링 한 후 함수로 재구성
    - 02_1번에서 수집된 데이터를 이용해서 여러 subpage 크롤링
    


In [20]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen




In [36]:

# 뉴스 sub 페이지

headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=009&aid=0004833477"
result = requests.get(url, headers=headers)


In [37]:
result


<Response [200]>

In [23]:
#result.content # 전송된소스코드를 반환 - row text 형식으로 반환 앞에 b 코드가 붙어있음
# result.text # 전송된 소스코드를 문자열로 반환

In [38]:
bs_obj = BeautifulSoup(result.content, "html.parser")
## 기사 제목 추출
title = bs_obj.find(id='articleTitle')
# title.text

## 기사 입력 시간
date_time = bs_obj.find("span",{"class":"t11"})
# date_time.text

## 좋아요 수 (동적으로 실시간으로 변해서 다른방법을 사용해야함 셀레니움 패키지 사용)
like_count = bs_obj.find("span",{"class":"u_likeit_text _count num"})
# bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")
like_count


#### 개발자 도구의 copy selector 사용방버

- 크롤링하려는 컨텐츠에서 검사 - 해당 소스에서 단축메뉴 - copy - copy selector
    - 붙여넣기 하면 선택자를 반환해주는데 선택된 내용이 없는 경우가 있음
        - 잘못된 선택자를 생성했을 경우 - 선택자 재구성
        - 동적으로 생성되는 태그인 경우에는 선택자가 만들어지지만 정적 크롤링 불가능
            - 네이버 뉴스 서브페이지의 좋아요 수와 기사 댓글 수는 동적 생성이되어서 정적 크롤링 불가능
            - 셀레니움이라는 패키지를 사용해야 함
            
bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")

In [25]:
# sub page 에서 수집된 데이터 반환하는  함수 작성

def get_news_sub_info(url) :

    # 1. 전달된 url에 접속 후 응답 객체 반환
    headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url, headers=headers)
    
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    # 3. 필요데이터 추출
    ## 기사 제목 추출 (태그객체)
    title = bs_obj.find(id='articleTitle')
    ## 기사 입력 시간 (태그객체)
    date_time = bs_obj.find("span",{"class":"t11"})
    
    #4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text
    dict_sub["기사 입력 시간"] = date_time.text
    
    return dict_sub

In [39]:
## main : 함수 호출 

headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com"
res = requests.get(url, headers=headers)
html = res.text

bs_obj = BeautifulSoup(html, "html.parser")

# 헤드라인 뉴스의 구성 ul , class : hdline_article_list

news_list = bs_obj.find("ul", {"class":"hdline_article_list"})

# news_list


lis = news_list.findAll("li")

link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])

    
# link_list
for url in link_list :
    res = get_news_sub_info(url)
    print(res)

{'기사제목': '연휴 하루전 고속도로 정체 본격화…오후 6~7시께 절정', '기사 입력 시간': '2021.09.17. 오후 5:34', '기사': '\n\n\n\n\n17일 오후 서울 서초구 잠원IC에서 바라본 경부고속도로 상(왼쪽)·하행선의 모습. 귀성 차량들로 하행선이 붐비고 있다. [연합][헤럴드경제=민성기 기자] 추석 연휴 전날인 17일 고속도로 정체가 본격화하고 있다.한국도로공사에 따르면 이날 오후 4시 기준 경부고속도로 부산 방향은 한남∼서초, 서울요금소∼신갈분기점 부근, 동탄분기점∼남서 부근, 천안 부근∼옥천 등 총 35㎞ 구간에서 차들이 제 속도를 내지 못하고 있다.서해안고속도로 목포 방향은 팔탄분기점 부근과 서평택부근∼서해대교 등 모두 10㎞ 구간에서, 서울 방향은 당진∼서해대교, 발안∼팔탄분기점, 비봉∼용담터널, 일직분기점∼금천 등 총 34㎞ 구간에서 정체가 발생했다.또 수도권제1순환고속도로 구리 방향은 판교분기점∼성남요금소, 자유로∼송내 등 30㎞ 구간에서, 일산 방향은 시흥요금소∼송내, 구리∼상일 등 26㎞ 구간에서 거북이 걸음이 이어지고 있다.중부내륙고속도로는 양양 방향 북충주∼충주분기점, 감곡 부근 등 10㎞ 구간과 창원 방향 여주분기점∼감곡 등 13㎞ 구간에서 가다 서기를 반복하고 있다.중부고속도로 하남 방향은 일죽∼모가 등 18㎞ 구간에서, 남이 방향은 대소∼진천터널 등 24㎞ 구간에서 서행 중이다. 영동고속도로는 강릉 방향 반월터널∼부곡 등 8㎞ 구간과 인천 방향 군자분기점∼서창분기점, 동수원∼광교터널 등 16㎞ 구간에서 느림보 운행이 이뤄지고 있다.전국 고속도로에는 이날 약 541만대의 차량 이동이 예상된다. 수도권에서 지방으로 47만대, 지방에서 수도권으로 48만대가 움직일 전망이다.귀성 행렬은 이날 오후 6~7시에 절정에 달해 오후 9~10시에서야 해소될 전망이다.이날 오후 4시 승용차로 서울 요금소를 출발해 전국 주요 도시까지 걸리는 예상 시간은 부산 5시간, 울산 4시간 40분, 대구 4시간, 광주 3시간 40분, 강

In [27]:
bs_obj


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/main/home.naver" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image">
<meta content="정치, 경제, 사회, 생활/문화, 세계, IT/과학 등 언론사별, 분야별 뉴스 기사 제공" property="og:description"/>
<meta content="네이버" property="og:article:author">
<meta content="summary" name="twitter:card"/>
<meta content="네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" name="twitter:image"/>
<meta content

#### 연습문제
- get_news_sub_info 함수의 반환겂에 기사 내용까지 포함되게 코드를 수정하고
- 반환되는 결과값을 df로 구성하시오.



In [28]:

def get_news_sub_info(url) :

    # 1. 전달된 url에 접속 후 응답 객체 반환
    headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url, headers=headers)
    
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    # 3. 필요데이터 추출
    ## 기사 제목 추출 (태그객체)
    title = bs_obj.find(id='articleTitle')
    ## 기사 입력 시간 (태그객체)
    date_time = bs_obj.find("span",{"class":"t11"})
    
    #기사 내용 추출(id=articleBodyContents)
    contents = bs_obj.find(id = "articleBodyContents")
    
    
    #4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text
    dict_sub["기사 입력 시간"] = date_time.text
    dict_sub["기사"] = contents.text
    return dict_sub

- 함수 끝


### main 시작

In [29]:
# 빈 데이터 프레임 하나 생성
import pandas as pd
dict_sub = {}
dict_sub["기사제목"] = []
dict_sub["기사 입력 시간"] = []
dict_sub["기사"] = []
news_sub = pd.DataFrame(dict_sub)
news_sub

,기사제목,기사 입력 시간,기사


In [30]:
## main : 함수 호출 
 
headers = {"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com"
res = requests.get(url, headers=headers)
html = res.text

bs_obj = BeautifulSoup(html, "html.parser")

# 헤드라인 뉴스의 구성 ul , class : hdline_article_list

news_list = bs_obj.find("ul", {"class":"hdline_article_list"})

# news_list


lis = news_list.findAll("li")

link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])

    
# link_list
for url in link_list :
    res = get_news_sub_info(url)
    # 빈 df에 반환되는 dict 행으로 추가 (append 함수 이용)
    news_sub = news_sub.append(res, ignore_index=True)

news_sub

,기사제목,기사 입력 시간,기사
0,"원세훈 '정치공작' 파기환송심, 2년 늘어난 징역 9년 선고",2021.09.17. 오후 2:32,\n\n\n\n\n 원세훈 전 국가정보원장(좌) [중앙포토] ...
1,"""반역자는 꺼져라""...박정희 생가 찾은 윤석열의 딜레마",2021.09.17. 오후 3:09,\n\n\n\n\n윤석열 국민의힘 대선 경선 후보가 17일 경북 구미시 박정희 전 ...
2,"금감원, '손태승 DLF 징계 취소' 항소하기로",2021.09.17. 오전 10:24,\n\n\n\n\n[서울=뉴시스] 류병화 기자 = 금융감독원이 손태승 우리금융지주 ...
3,"文 대통령 ""3600만명 1차접종 완료…단계적 일상 회복 희망 앞당겨져""",2021.09.17. 오후 5:16,\n\n\n\n\n[이미지출처=연합뉴스]문재인 대통령이 코로나19 백신 1차 접종자...
4,코로나19 백신 1차 접종 70% 달성…접종 시작 후 204일만(상보),2021.09.17. 오후 5:39,\n\n\n\n\n오후 5시 기준 3600만4101명 접종…18세 이상 기준으로는 ...


In [31]:
### 기존에 수집한 데이터

In [32]:
news_df = pd.read_csv("./crawl_data/naver_news.csv", index_col = 0)

In [33]:
news_df

,section,news_title,link_url
0,헤드라인,"'70% 접종' 달성에도 심상찮은 확산세…""추석 잘 넘겨야 일상회복""",https://news.naver.com/main/read.naver?mode=LS...
1,헤드라인,"'참 멍멍이OO 같네' 택배노조 집단 괴롭힘…점주 아내, 13명 고소",https://news.naver.com/main/read.naver?mode=LS...
2,헤드라인,홍준표 “국민들이 조국 과잉 수사 아니라고 생각한다면 제 생각 바꿀 수밖에”,https://news.naver.com/main/read.naver?mode=LS...
3,헤드라인,문 닫거나 반쪽 되는 코인거래소…현재 전체 5∼7% 거래,https://news.naver.com/main/read.naver?mode=LS...
4,헤드라인,"""복학 안 돼 암담""... '청년 문재인'도 일자리 고민 컸다",https://news.naver.com/main/read.naver?mode=LS...
5,정치,"이준석 ""文지지율 40%? 이례적인 것 아냐..과감히 정치개혁""",https://news.naver.com/main/read.naver?mode=LS...
6,정치,"이재명 ""주인 왜 내게 묻나""…이낙연 ""진실 규명돼야""",https://news.naver.com/main/read.naver?mode=LS...
7,정치,[여랑야랑]돌직구형? 역공형? 국민의힘 토론스타일 / 윤석열도 홍준표도 ‘봉변’…토...,https://news.naver.com/main/read.naver?mode=LS...
8,정치,커지는 ‘조국수홍’ 후폭풍…국민의힘 지지층서 비판,https://news.naver.com/main/read.naver?mode=LS...
9,정치,"이재명 ""한 톨 먼지라도 있으면 가루"" vs ""상식적이지 않다""",https://news.naver.com/main/read.naver?mode=LS...


In [34]:
# 빈 데이터프레임 하나 생성
import pandas as pd
dict_sub={} # 빈 dict
dict_sub["기사제목"] = [] #dict에 원소 생성
dict_sub["기사 입력 시간"]=[]
dict_sub["기사"] = []
news_sub = pd.DataFrame(dict_sub) #df를 생성
news_sub


,기사제목,기사 입력 시간,기사


In [35]:
import time

link_list = news_df["link_url"]

for url in link_list :
    res = get_news_sub_info(url)
    # 빈 df에 반환되는 dict 행으로 추가 (append 함수 이용)
    news_sub_main = news_sub_main.append(res, ignore_index=True)
    time.sleep(2) # 계속되는 요청으로 서버가 disconnect 시킬 수 있으므로 시간 간격을 둔다.

news_sub_main

NameError: name 'news_sub_main' is not defined